In [1]:
import pandas as pd
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
df = pd.read_csv(outputFilePath+'user_category'+".csv")
print('end')

end


In [4]:
import numpy as np
import threading
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix


def do_dot(a, b, outIn, rowStartInd, blockSize, nBlocks, outputFilePath):
    nRows = a.shape[0]
    filecount = 1
    outputFilePrefix = 'user_user_' + str(outIn) + "_"
    writefile= open(outputFilePath+outputFilePrefix+"1.txt","w+")
    header = 'user_index@most_similar_user_index@value\n'
    writefile.writelines(header)

    for i in range(0, nRows):
        maxInd =0
        maxVal =0
        count = i + 1
        for j in range(0 , nBlocks):
            if j == 0:
                blockStartInd = j*blockSize
            else:
                blockStartInd = (j*blockSize) + 1

            blockEndInd = (j+1)*blockSize
            blockShapeA = csr_matrix(a[i]).shape
            blockShapeB = csr_matrix(b[blockStartInd:blockEndInd,:]).shape
            if(blockShapeA[0]!=0 and blockShapeB[0]!=0):
                res = cosine_similarity(csr_matrix(a[i]), csr_matrix(b[blockStartInd:blockEndInd ,:]))
                actualInd = (rowStartInd + i)
                
                if actualInd in range(blockStartInd,blockEndInd+1):
                    res[0, i] = 0
 
                #tempMaxInd = np.argsort(res.flatten())[-2]
                #tempMaxVal = res.flatten()[tempMaxInd] #tempMaxVal = res.max()
                tempMaxInd = np.argmax(res[0])
                tempMaxVal = res[0, tempMaxInd]
                tempMaxInd = blockStartInd - 1 + tempMaxInd
                if(tempMaxVal> maxVal):
                    maxInd = tempMaxInd
                    maxVal = tempMaxVal
        if count == (filecount * 100000): # change to new file for every 1 lakh rows
            writefile.close()
            filecount = filecount+1
            writefile= open(outputFilePath+outputFilePrefix+str(filecount)+".txt","w+")       
            writefile.writelines(header)
        writefile.writelines(str(rowStartInd+i)+"@"+str(maxInd)+"@"+str(maxVal)+"\n")
        print(str(outIn)+"_"+str(i)+"@"+str(maxInd)+"@"+str(maxVal))

    writefile.close()
    return

def pardot(b, rowChunkSize, nThreads, blockSize, nBlocks,outputFilePath, dot_func = do_dot):

    threads = []

    a = b
    for i in range(1, nThreads):
        if i == 0:
            rowStartInd = i*rowChunkSize
        else:
            rowStartInd = (i*rowChunkSize) + 1
        rowEndInd = (i+1) * rowChunkSize
        newMatrix1 = a[rowStartInd:rowEndInd]
        newMatrix2 = a
        th = threading.Thread(target = dot_func, args=(newMatrix1,newMatrix2, i, rowStartInd, blockSize, nBlocks, outputFilePath ))
        th.setDaemon(False)
        th.start() 
        threads.append(th)
        
    for th in threads:
        th.join()
    return

if __name__ == '__main__':
    vals = df.drop(['user_id'], axis = 1).to_numpy(dtype= 'float32')
    vals = vals[0:1637000, :]
    print(vals.shape)
    start = time()
    # nRows = 1637138
    rowChunkSize = 1000 #10000
    nThreads = 2 # 163
    blockSize = 1000
    nBlocks = 1637
    outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/threads/"
    
    pardot(vals, rowChunkSize, nThreads, blockSize, nBlocks, outputFilePath)
    time_par = time() - start
    print('pardot: {:.2f} seconds taken'.format(time_par))
    

12959311796411741


112276
511644
212000
12960
112277
311797
212001411742

511645
12961


KeyboardInterrupt: 

In [3]:
import numpy as np
import threading
from time import time 

def compute (outputInd, rowStartInd, rowEndInd, vals, user_id_dict):
    filecount = 1
    outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
    outputFilePrefix = 'user_popular_'+ str(outputInd)
    writefile= open(outputFilePath+outputFilePrefix+"1.txt","w+")
    header = 'FileIndex@user_index@user_id@most_similar_users\n'
    writefile.writelines(header)

    # for i in range(0, vals.shape[0]):
    for i in range(rowStartInd, rowEndInd):
        count = i+1
        print(i)
        topCategories = pd.Series(vals.iloc[i,:]).nlargest(5)
        #print(topCategories) 
        if count == (filecount * 100000): # change to new file for every 1 lakh rows
            writefile.close()
            filecount = filecount+1
            print("count: "+ str(count) + " curr filecount: "+ str(filecount))
            writefile= open(outputFilePath+outputFilePrefix+str(filecount)+".txt","w+")       
            writefile.writelines(header)
        if(count%1000 ==0):
            print("count:"+str(count)+str(outputInd)+"@"+str(i+rowStartInd)+"@"+user_id_dict[i+rowStartInd]+"@"+ str(user_ids))
        # find top 3 users in each category - most popular recommendations
        userIndices = vals.nlargest(3, topCategories.index).index.values.tolist()
        user_ids = [user_id_dict[x] for x in userIndices]
        writefile.writelines(str(outputInd)+"@"+user_id_dict[i]+"@"+ str(user_ids)+"\n")
        print(str(outputInd)+"@"+str(i+rowStartInd)+"@"+user_id_dict[i+rowStartInd]+"@"+ str(user_ids))
    writefile.close()
    return

def createThreads(a, nThreads, rowChunkSize, rowPartitionStartInd, user_id_dict, dot_func = compute):
    threads = []
    matr = a
    
    for i in range(0, nThreads):
        if i == 0:
            rowStartInd = rowPartitionStartInd + i*rowChunkSize
        else:
            rowStartInd = rowPartitionStartInd + (i*rowChunkSize) + 1
        rowEndInd = (i+1) * rowChunkSize

        th = threading.Thread(target = dot_func, args=(i, rowStartInd, rowEndInd, matr, user_id_dict ))
        th.setDaemon(False)
        th.start() 
        threads.append(th)
        
    for th in threads:
        th.join()
    return
    
     
    user_id_dict = df['user_id'].to_dict()
    vals = df.drop(['user_id'], axis = 1)
    print(vals.shape)
    start = time()
    rowChunkSize = 100000 #100000
    nThreads = 6 # 16
    #rowPartitionStartInd = 100000
    outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/threads/"
    createThreads(vals,nThreads, rowChunkSize, rowPartitionStartInd, user_id_dict)
    time_par = time() - start
    print('createThreads: {:.2f} seconds taken'.format(time_par))
    
print('end')  

end
112262511629

211986
411727
311781
12944
211987
511630
311782
112263


In [61]:
compute ("", 0, 1000, vals, user_id_dict)

In [ ]:
5:54 16 done , 75